In [1]:
import plotly as py
import plotly.graph_objs as go
import ipywidgets as widgets
import numpy as np

from math import log
from statistics import geometric_mean

py.offline.init_notebook_mode(connected=True)

In [2]:
import pandas as pd

from parsecsv import parse_csv

In [3]:
MIN_NGS_ADDED = 1
BASE = "base"
BASE_HEUR = "base-heur"
TIMEOUT = 900
PROCESSING = "first"

LIMS = {}
LIMS["ground"] = (20, 900)
LIMS["time"] = (0.1, TIMEOUT*1.5)
LIMS["ctime"] = (0.1, TIMEOUT*1.5)


LIMS["csolve"] = (0.1, 900)
LIMS["mem"] = (5_000, 16_800_000)
LIMS["choices"] = (1, 1_000_000_000)
LIMS["conflicts"] = (1, 1_000_000_000)


In [4]:
def get_dom(instance):
	inst = instance.split("/")

	if "ipc-" in instance:
		#return inst[-4] + "-" + inst[-2]
		return "rintanen"

	if "hard" in instance:
		#return inst[-4] + "-" + inst[-3]
		return "hard"
		
	elif "easy" in instance:
		#return inst[-4] + "-" + inst[-3]
		return "easy"
	
	elif "instance" in instance:
		return inst[-3]
	
	elif "asp" in instance:
		return inst[-2]

	raise RuntimeError(f"No dom for instances {instance}")


In [16]:
def plotly_scatter(dataframes, config, stat, min_ngs, min_choices):
    if "heur" in config:
        base_config = "base-heur"
    else:
        base_config = "base"
    
    instances_ngadded = dataframes["ngadded"][config][ (dataframes["ngadded"][config] >= min_ngs) & \
                                                                        ( (dataframes["time"][config] != TIMEOUT) & \
                                                                        (dataframes["time"][base_config] != TIMEOUT) & \
                                                                        (dataframes["choices"][base_config] >= min_choices))].index

    print(config)
    print(f"Total instances: {len(instances_ngadded)}")

    frame = pd.DataFrame()
    frame["x"] = dataframes[stat][base_config][instances_ngadded]
    frame["y"] = dataframes[stat][config][instances_ngadded]

    frame["dom"] = [get_dom(inst) for inst in instances_ngadded]
    frame["ngadded"] = [int(ng) for ng in dataframes["ngadded"][config][instances_ngadded]]
    
    layout = go.Layout(
        title=dict(
            text=f"Scatter plot - {config} <br> Minimum of {min_ngs} nogoods <br> {len(frame['x'])} instances",
            y=0.9,
            x=0.5,
            xanchor="center",
            yanchor="top"
        ),
        yaxis=dict(
            title=f"{stat} with config {config}",
            type="log",
            range=[log(v,10) for v in LIMS[stat]]
        ),
        xaxis=dict(
            title=f"{stat} with base",
            type="log",
            range=[log(v,10) for v in LIMS[stat]]
        ),
        width=1400,
        height=1000,
    )
    

    grouped = frame.groupby("dom")

    data = []
    for dom in sorted(set(frame["dom"])):
        df = grouped.get_group(dom)
        base_gmean = geometric_mean([float(v)+0.1 for v in df['x']])
        base_avg = df['x'].mean()
        config_gmean = geometric_mean([float(v)+0.1 for v in df['y']])
        config_avg = df['y'].mean()
        ng_avg = df["ngadded"].mean()
        print(f"{stat:10}  :  {dom:50}  :  {len(df.index):4}  :  base : {base_gmean:12.3f} : base avg  : {base_avg:10.2f} config: {config_gmean:12.3f} : config avg  : {config_avg:10.2f}  :  avg ng added: {ng_avg:12.2f}")
        
        trace = go.Scatter(
            x=df["x"],
            y=df["y"],
            mode="markers",
            name=f"{dom}",
            marker=dict(
                cmax=1500,
                cmin=0,
                color=df["ngadded"],
                colorbar=dict(
                    title="Added nogoods",
                    y=1,
                    x=-0.2,
                    ticks="outside",
                    yanchor="top",
                    
                ),
                colorscale="Viridis"
            )    
        )
        
        data.append(trace)
        
    fig = go.Figure(data=data, layout=layout)

    if stat in LIMS:
        limits = LIMS[stat]
        fig.add_shape(type="line",x0=0,y0=0, x1=limits[1],y1=limits[1],
                line=dict(color='Red'),
                xref='x',
                yref='y')

    py.offline.iplot(fig)

In [17]:
# These 2 files have as the base the modified encoding WITHOUT adding temporal nogoods
#file_name = "incremental/results-generalization-inc-asp.csv"
file_name = "incremental/results-generalization-inc-pddl.csv"
#file_name = "incremental/results-generalization-inc-pddl-rintanen.csv"

# These 2 files have as the base the NORMAL incremental encoding
# If you use these 2 files, set the minimim nogoods added to 0
#file_name = "incremental/results-generalization-inc-asp-vs-normal.csv"
#file_name = "incremental/results-generalization-inc-pddl-vs-normal.csv"

dataframes, merged_df, config_names, stat_names = parse_csv.read_csv(file_name, do_processing=PROCESSING)


In [18]:
layout = widgets.Layout(width='400px')

min_ngs = widgets.BoundedIntText(min=0, max=1000, value=1, step=10, description="Minimum nogoods added")
min_choices = widgets.BoundedIntText(min=0, max=500000, value=0, step=1000, description="Minimum choices to consider in base solve")


non_base_configs = [c for c in set(config_names) if "base" not in c]
config = widgets.Select(options=sorted(non_base_configs), value=sorted(non_base_configs)[0], description="Configuration", layout=layout)

stat = widgets.Select(options=["time", "ctime", "conflicts", "choices"], value="choices", description="Stats")

widgets.interactive(lambda config, stat, min_ngs, min_choices: plotly_scatter(dataframes, config, stat, min_ngs, min_choices), \
                    config=config, stat=stat, min_ngs=min_ngs, min_choices=min_choices)



interactive(children=(Select(description='Configuration', layout=Layout(width='400px'), options=('max-1000-ste…